In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [1]:
class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wb), np.zeros_like(b)]
        self.cache = None
        
    def forward(self, x, h_prev):
        Wx, Wh, b = self.parms
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
        h_next = np.tanh(t)
        
        self.cache = (x, h_prev, h_next)
        # 역전파 계산시 사용하는 중간 데이터 담는 곳.
        return h_next
    
    def backward(self, dh_next):
        Wx, Wb, b = self.params
        x, h_prev, h_next = self.cache
        
        dt = dh_next * (1 - h_next ** 2)
        db = np.sum(dt, axis = 0)
        dWh = np.matmul(h_prev.T, dt)
        dh_prev = np.matmul(x.T, dt)
        dx = np.matmul(dt, Wx.T)
        
        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

In [ ]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        # 다수의 RNN 계층을 list에 저장하는 용도
        self.layers = None
        
        self.h, self.dh = None, None
        self.stateful = stateful
        
    def set_state(self, h):
        # h : 은닉상태 저장.
        self.h = h
    
    def reset_state(self):
        self.h = None

        
    def forward(self, xs):
        Wx, Wh, b = self.params
        # N : 미니배치 크기 / T : T개 시계열 데이터 / D : 입력 벡터 차원 수
        N, T, D = xs.shape
        D, H = Wx.shape
        
        self.layers = []
        hs = np.empty(N, T, H), dtype='f')
        
        # 첫 호출시, 은닉상태는 0행렬로 초기화.
        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')
            
        for t in range(t):
            layer = RNN (* self.params)
            self.h = layer.forward(xs[:, t, :], self.h)
            hs[:, t, :] = self.h
            self.layers.append(layer)
            
        return hs
            
    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D, H = Wx.shape
        
        dxs = np.empty((N, T, D), dtype='f')
        dh = 0
        grads = [0, 0, 0] # dWx, dWh, db 순
        for t in reversed(range(T)):
            layer = self.layer[t]
            dx, dh = layer.backward(dhs[:, t, :] + dh)
            dxs[:, t, :] = dx
        
            for i, grad in enumerate(grads):
                grads[i] += grad
            
        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh            
        
        return dxs